# Checking Kaustav's conjecture using gradient descent

In this notebook I numerically check that a certain conjecture in matrix theory holds true, here referred to as Kaustav's conjecture. The conjecture arises from the problem of maximizing orbital correlations in so-called fermionic 'free' quantum states over unitary rotations of the underlying orbital space.

## 1. The conjecture

Our conjecture concerns the spectra of diagonal blocks of square matrices. Let $d$ and $\lambda$ be natural numbers satisfying $d \geq 2$ and $d/2 \leq \lambda < d$. Consider a vector
$$
\boldsymbol{n} = (n_1, \ldots, n_d) \in \mathbb{R}^d
$$
such that $1 \geq n_1 \geq \ldots \geq n_d \geq 0$ (these requirements are the Pauli constraints for fermionic occupation numbers, and play a minor role in the present mathematical treatment). Let $N$ be a Hermitian matrix with spectrum $\boldsymbol{n}$, i.e., equivalently, consider an arbitrary $d \times d$ unitary matrix $U$ and set
$$
N = U \cdot \text{diag}(\boldsymbol{n}) \cdot U^\dagger \; .
$$
We now focus on the upper-left $\lambda \times \lambda$ diagonal block $B_{11}$ of $M$ and on its lower-right $(d - \lambda) \times (d - \lambda)$ diagonal block $B_{22}$:
$$
N = 
\begin{pmatrix}
B_{11} & * \\
* & B_{22}
\end{pmatrix} \; .
$$

Kaustav's conjecture is about the relation of the spectrum $\boldsymbol{n}$ of $N$ and the spectra of $B_{11}$ and $B_{22}$. To formulate the conjecture, we still need a bit of notation. We pack the two spectra of the diagonal blocks in the single `$\lambda$-block spectrum'
$$
 \boldsymbol{b} = \boldsymbol{b}_\lambda(N) = ( \underbrace{b_1, \ldots, b_\lambda}_{\text{spectrum of }B_{11}}, \underbrace{b_{\lambda + 1}, \ldots, b_d}_{\text{spectrum of }B_{22}}  ) \; ,
$$
where, just to be clear, each of the two spectra is ordered decreasingly. Also, we set
$$
\boldsymbol{b}^K_\lambda(\boldsymbol{n}) = \left (\frac{n_1 + n_d}{2}, \frac{n_1 + n_d}{2}, \frac{n_2 + n_{d-1}}{2},\frac{n_2 + n_{d-1}}{2},  \ldots, \frac{n_{d - \lambda} + n_{\lambda + 1}}{2}, n_{d - \lambda + 1}, \ldots , n_{\lambda}  \right ) \; .
$$

Importantly, the vector $\boldsymbol{b}^K_\lambda(\boldsymbol{n})$ itself can be pictured (up to a permutation of its components) as a $\lambda$-block spectrum $\boldsymbol{b} = \boldsymbol{b}_\lambda(N)$ of a matrix $N$ with spectrum $\boldsymbol{n}$! Just to give the rough idea behind this fact, consider the simple case $d = 2$ and $\lambda = 1$, and set
$$
U = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix} \; .
$$
This (Hadamard) matrix in some sense maximally reshuffles the components of any vector it is applied to. As a somewhat related fact, it is straightforward to check that the $\lambda$-block spectrum (i.e., the diagonal) of the matrix $ N = U \cdot \text{diag}(n_1, n_2) \cdot U^\dagger$ is indeed given by
$$
\boldsymbol{b} = \boldsymbol{b}_\lambda(N) = \boldsymbol{b}^K_\lambda(\boldsymbol{n}) = \left (\frac{n_1 + n_2}{2}, \frac{n_1 + n_2}{2} \right ) \; .
$$
The general argument to show that $\boldsymbol{b}^K_\lambda(\boldsymbol{n})$ can be pictured as a $\lambda$-block spectrum $\boldsymbol{b}$ for any $d$ and $\lambda$ is obtained by generalizing the above construction.

We are now ready to state Kaustav's conjecture.

**Kaustav's conjecture:**
For given $d$, $\lambda$ and $\boldsymbol{n} \in [ 0,1 ]^d$, let $B$ be the set of all $\lambda$-block spectra $\boldsymbol{b}$ of Hermitian matrices with spectrum $\boldsymbol{n}$. Then the vector $\boldsymbol{b}^K_\lambda(\boldsymbol{n})$ (which is in $B$, up to a permutation of components) is majorized by any other $\boldsymbol{b} \in B$.

This conjecture is reminiscent of the Schur-Horn theorem and related problems (see the [blog by Terence Tao](https://terrytao.wordpress.com/2010/01/12/254a-notes-3a-eigenvalues-and-sums-of-hermitian-matrices/#more-3341) for an introduction). Also, for more analytical insights, check out [this reference](https://faculty.sites.iastate.edu/ytpoon/files/inline-files/26.pdf) by Chi-Kwong Li and Yiu-Tung Poon for a characterization of the set $B$. Finally, [this work](https://michaelnielsen.org/papers/maj-book-notes.pdf) by Michael Nielsen introduces majorization.

An interesting (for our research) consequence of the above conjecture is that any Schur-concave function would be maximized by $\boldsymbol{b}^K_\lambda(\boldsymbol{n})$ over $B$, by definition of Schur-concavity. The goal of this project is to numerically check that this is the case for the Schur-concave entropic function
$$
H(\boldsymbol{b}) = \sum_{i = 1}^d h(b_i)  \;,
$$
where $h(x) = - x \ln(x) - (1 - x) \ln ( 1 - x ) $ is the binary entropy. At the same time, we will also carry our explicit checks of the main conjecture. (Note: the reason why we restricted to $\boldsymbol{n} \in [0,1]^d$ was just that $h$ is defined on $[0,1]$, but the main conjecture can and should be formulated for arbitrary $\boldsymbol{n} \in \R^d$!) More precisely, $h$ is strictly concave and $H$ is thus strictly Schur-concave; and as a result, verifying Kaustav's conjecture would imply that $\boldsymbol{b}^K_\lambda(\boldsymbol{n})$ is the unique maximizer of $H$ over $B$.

We view and perform the maximization of $H$ over the set $B$ as an optimization over the $d \times d$ unitary matrix $U$, as explained in the next paragraph.


## 2. Numerically testing the conjecture with a gradient descent method

Let us now turn to a numerical approach. In the `src` directory, we implemented code that performs the optimization described above. In detail, we (redundantly) parametrize the set of $d \times d$ unitary matrices $U$ with the $d^2$-dimensional $\mathbb{R}$-vector space of $d \times d$ anti-Hermitian matrices $A$, by exponentiation, i.e., $U = \exp A$. The antihermitian matrices $A$ are easily recast as $d \times d$ matrices $M$ with real entries (see function `M_to_A` in `src/utils.py`). These matrices $M$ are handled using `torch`, which allows for gradient descent methods. For given $\lambda$ and a given vector $\boldsymbol{n} \in [0, 1]^d$, we minimize over $M$ the cost function $M \mapsto C(M)$, computed as follows:
\begin{equation}
    M \mapsto A = \text{M\textunderscore to\textunderscore A}(M) \mapsto U = \exp (A) \mapsto N = U \text{diag}(\boldsymbol{n}) U^\dagger \mapsto \boldsymbol{b} = \boldsymbol{b}_\lambda(N) \mapsto C(M) = - H(\boldsymbol{b}) \; .
\end{equation}
The function `build_cost_function` in `src/core.py` builds the above cost function, i.e., it accepts as args an integer $\lambda$ and a vector $\boldsymbol{n} \in [0, 1]^d$, and returns the cost function.

The actual optimization is performed by the function `get_b_best` in `src/core.py`. This function also accepts as args $\lambda$ and $\boldsymbol{n}$, along with some optional parameters to tweak the gradient descent method. It then builds the corresponding cost function by calling `build_cost_function`. That cost function is then minimized over $M$ via a gradient descent method, using the `torch` class `torch.optim.Adam`. During the optimization, data of interest gets printed. The main results, which are also returned as output by `get_b_best`, are an optimized unitary $U$, the corresponding block spectrum $\boldsymbol{b}$, the maximized value $H(\boldsymbol{b})$, and a boolean value `conjecture_holds` which keeps track of possible violations to Kaustav's conjecture (regarding either the conjectured majorization relation, or the expected maximum of $H$). More details are explained in the function description, see `src/core.py`. The next paragraphs of this notebook present some examples.

## 3. A simple scenario: $d = 2, \lambda = 1$

First, we import the necessary libraries and our implementation of the gradient descent optimization.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
import os

# Add the src directory to the path to import our custom module
sys.path.insert(0, os.path.join('..', 'src'))

# Import our functions
from kaustav_conj.utils import bK
from kaustav_conj.core import get_b_best

We start with the simplest case, namely $d = 2$ and $\lambda = 1$, and a single vector $\boldsymbol{n} = (n_1, n_2)$. For this case, Kaustav's conjecture predicts the optimal block spectrum to be $\left( \frac{n_1 + n_2}{2}, \frac{n_1 + n_2}{2} \right)$, which can be obtained, e.g., by setting $U$ to be a Hadamard rotation (see above).

In [2]:
n = [0.2, 0.4]
lamb = 1
bK(n, lamb)

array([0.3, 0.3])

We now call the optimizer `get_b_best`. We carry out a single gradient descent, i.e., we consider only one initialization (`N_init=1`).

In [3]:
U_best, b_best_num, H_best, conjecture_holds = get_b_best(n, lamb, N_init=1, N_steps=150,learning_rate=0.01)


Starting get_b_best optimization for n = [0.2, 0.4]
Parameters recap:
  n = [0.2, 0.4]
  lambda = 1
  rand_range = 1.0
  N_init = 1
  N_steps = 150
  learning_rate = 0.01
  eps = 1e-12
  print_more = False

Starting gradient descent run 1/1

Printing final results, optimized over all N_init = 1 initializations
Numerical b_best = [0.30000161 0.29999839]
Conjectured b_best = [0.3 0.3]
Norm of difference = 2.2743255729868446e-06
Conjectured H_best - numerical H_best (should be > 0) = 1.2315481967561936e-11
Conjectured majorization = True
No violations to the conjecture were found.

Finished get_b_best optimization for n = [0.2, 0.4]



The results agree with Kaustav's conjecture. In particular, notice that the `Norm of difference` value, i.e. the norm distance between the conjectured optimal block spectrum `bK(n, lamb)` and the numerical result for it, is negligible (and vanishes up to numerical precision if `Nsteps` and `learning_rate` are tweaked properly). Finally, we check that the optimal unitary is compatible with a Hadamard gate, up to phases:

In [4]:
print(torch.abs(U_best).data) # all entries should be 1/sqrt(2)

tensor([[0.7071, 0.7071],
        [0.7071, 0.7071]], dtype=torch.float64)


## 4. Testing the conjecture

By using a cluster, it was checked that Kaustav's conjecture holds for any choice of $d$ and $\lambda$. Specifically, >10 vectors $\boldsymbol{n}$ were tested for each pair $(d, \lambda)$ for $d$ up to 18 and any allowed $\lambda$. Also, in total, about 50 more vectors $\boldsymbol{n}$ were tested for pairs $(d, \lambda)$ with high $d$ (up to $d = 100$). A violation to Kaustav's conjecture was never found. Most importantly, all of these computations converged to the conjectured values. In particular, we checked convergence in norm of the numerically obtained optimal block spectrum to the conjectured value `bK(n, lamb)`.

The next cell shows an example of the code used for our tests (see also `python_sampling_scripts/check_kaustav_conj.py`).

In [6]:
d = 5
N_n = 1 # number of n vectors per choice of d and lambda
N_init = 5 # number of initializations per optimization
N_steps = 300
learning_rate = 0.01
eps = 1.e-12
conjecture_holds = True
for lamb in range(int(np.ceil(d/2)), d):
    print(F"\n{'='*40}\n{'='*40}\nCASE d = {d}, lambda = {lamb}\n{'='*40}\n{'='*40}\n")
    for _ in range(N_n):
        n = np.random.rand(d)
        b_best_conj = bK(n, lamb)
        U_best, b_best_num, H_best, conjecture_holds = get_b_best(n, lamb, N_init=N_init, N_steps=N_steps,learning_rate=learning_rate, eps=eps)
        if conjecture_holds == False:
            break
    if conjecture_holds == False:
        break

print(f"\n{'='*40}\n{'='*40}\nConjecture holds: {conjecture_holds}")
if not conjecture_holds:
    print(f"!!!!! COUNTEREXAMPLE FOUND within threshold for inequality checks of = {eps}. Exiting code before time.")
    print(f"n: {n}")
    print(f"U_best: {U_best}")
    print(f"b_best_num: {b_best_num}")
print(f"{'='*40}\n{'='*40}\n")


CASE d = 5, lambda = 3


Starting get_b_best optimization for n = [4.77584482e-01 5.74628927e-01 8.41075536e-04 3.48869884e-02
 9.07831777e-01]
Parameters recap:
  n = [4.77584482e-01 5.74628927e-01 8.41075536e-04 3.48869884e-02
 9.07831777e-01]
  lambda = 3
  rand_range = 1.0
  N_init = 5
  N_steps = 300
  learning_rate = 0.01
  eps = 1e-12
  print_more = False

Starting gradient descent run 1/5
Starting gradient descent run 2/5
Starting gradient descent run 3/5
Starting gradient descent run 4/5
Starting gradient descent run 5/5

Printing final results, optimized over all N_init = 5 initializations
Numerical b_best = [0.47758448 0.45433648 0.30475766 0.45433638 0.30475825]
Conjectured b_best = [0.47758448 0.45433643 0.30475796 0.45433643 0.30475796]
Norm of difference = 4.246643818255299e-07
Conjectured H_best - numerical H_best (should be > 0) = 2.9417432934053522e-09
Conjectured majorization = True
No violations to the conjecture were found.

Finished get_b_best optimization for n 